In [84]:
import pandas as pd
import re

from functools import partial

In [2]:
df = pd.read_parquet('./../data/parsed/kiids_parsed.parquet')

In [3]:
df

filename  \
0  KIID_MIL25_2022-02-21.pdf   

                                            raw_text  \
0  Kluczowe informacje dla Inwestorów \n\nNiniejs...   

                                               intro  \
0  Kluczowe informacje dla Inwestorów \n\nNiniejs...   

                        cele i polityka inwestycyjna  \
0  CELE I POLITYKA INWESTYCYJNA \n\nCelem  inwest...   

                               profil ryzyka i zysku  \
0  PROFIL RYZYKA I ZYSKU \n\nNiższe ryzyko\n\nWyż...   

                                              opłaty  \
0  OPŁATY \n\nJednorazowe  opłaty  pobierane  prz...   

                     wyniki osiągnięte w przeszłości  \
0  WYNIKI OSIĄGNIĘTE W PRZESZŁOŚCI \n\nRoczne sto...   

                               informacje praktyczne  
0  INFORMACJE PRAKTYCZNE \n\nDepozytariuszem Subf...

In [12]:
paragraph = df.iloc[0,3]

In [27]:
p_marked = re.sub('\n\n', '<p>', paragraph)
p_trimmed = re.sub('\s+', ' ', p_marked)
p_fixed = re.sub('<p>', '\n', p_trimmed)

# Find funds info

In [214]:
doc_text = df.raw_text[0]

In [215]:
knf_df = pd.read_pickle('./../data/knf_funds.pkl')

In [216]:
knf_df.shape

(1390, 6)

In [228]:
knf_df.head()

fundusz  \
0  ABELIA CAPITAL Fundusz Inwestycyjny Zamknięty ...   
1  AD ROYAL Fundusz Inwestycyjny Zamknięty Aktywó...   
2  AGIO Nieruchomości Komercyjnych Fundusz Inwest...   
3             AGIO PLUS Fundusz Inwestycyjny Otwarty   
4             AGIO PLUS Fundusz Inwestycyjny Otwarty   

                   subfundusz indentyfikator_krajowy nr_w_rejestrze  \
0                        None            PLFIZ000030           1212   
1                        None            PLFIZ000572            420   
2                        None            PLFIZ000037            986   
3                        None            PLFIO000281           1309   
4  Subfundusz AGIO Akcji PLUS            PLFIO000264           1309   

                                         towarzystwo  \
0  AgioFunds Towarzystwo Funduszy Inwestycyjnych ...   
1  AgioFunds Towarzystwo Funduszy Inwestycyjnych ...   
2  AgioFunds Towarzystwo Funduszy Inwestycyjnych ...   
3  AgioFunds Towarzystwo Funduszy Inwestycyjnych ...   
4  AgioFunds Towarzystwo Funduszy Inwestycyjnych ...   

  identyfikator_krajowy_funduszu  
0                    PLTFI000036  
1                    PLTFI000036  
2                    PLTFI000036  
3                    PLTFI000036  
4                    PLTFI000036

In [218]:
is_this = df_1['subfundusz'].apply(lambda text: re.search(f'{text}' , doc_text))

In [219]:
df_1[~is_this.isna()]

fundusz  \
19            Millennium Fundusz Inwestycyjny Otwarty   
85  Allianz Strategiczny Specjalistyczny Fundusz I...   

                         subfundusz indentyfikator_krajowy nr_w_rejestrze  \
19  Subfundusz Obligacji Globalnych            PLSFIO00002            182   
85             Subfundusz Obligacji            PLSFIO00089            614   

                                          towarzystwo identyfikator_krajowy  
19  MILLENNIUM Towarzystwo Funduszy Inwestycyjnych...           PLTFI000011  
85  Towarzystwo Funduszy Inwestycyjnych Allianz Po...           PLTFI000013

In [220]:
re.search('182', doc_text)

<re.Match object; span=(712, 715), match='182'>

In [221]:
intro_text = df.intro[0]


# Add data

In [222]:
does_not_id_exist = knf_df['nr_w_rejestrze'].apply(lambda register_id: re.search(str(register_id) , intro_text)).isna()
df_filtered = knf_df[~ does_not_id_exist]

df_filtered['nr_w_rejestrze'].iloc[0]

182

In [223]:
knf_df['fundusz'].sort_values(key=lambda x: x.str.len(), ascending=False)

9     EQUES MICHAEL STRӦM CREDITUM Fundusz Inwestycy...
9     Bonus Wierzytelności 2 Pragma Inkaso Niestanda...
10    Bonus Wierzytelności 3 Pragma Inkaso Niestanda...
23    Open Finance Wierzytelności Detalicznych Niest...
17    Getback Windykacji Platinum Niestandaryzowany ...
                            ...                        
22                     BPS Fundusz Inwestycyjny Otwarty
21                     BPS Fundusz Inwestycyjny Otwarty
19                     BPS Fundusz Inwestycyjny Otwarty
20                     BPS Fundusz Inwestycyjny Otwarty
5                         LL FIZ Aktywów Niepublicznych
Name: fundusz, Length: 1390, dtype: object

In [224]:
def retrive_register_id(knf_df: 'pd.DataFrame', intro_text: str) -> int:
    
    does_not_id_exist = knf_df['nr_w_rejestrze'].apply(lambda register_id: re.search(str(register_id) , intro_text)).isna()
    df_filtered = knf_df[~ does_not_id_exist]

    if len(df_filtered['nr_w_rejestrze'].unique()) > 1:
        return None
    
    return df_filtered['nr_w_rejestrze'].iloc[0]

In [225]:
def retrive_fund_name(knf_df: 'pd.DataFrame', intro_text: str) -> str:

    does_not_name_exist = knf_df['fundusz'].apply(lambda register_id: re.search(str(register_id) , intro_text)).isna()
    df_filtered = knf_df[~ does_not_id_exist]

    fund_name = df_filtered['fundusz'].sort_values(key=lambda x: x.str.len(), ascending=False).iloc[0]

    return fund_name
   

In [226]:
def retrive_subfund_name(knf_df: 'pd.DataFrame', intro_text: str) -> str:

    does_not_name_exist = knf_df['subfundusz'].apply(lambda register_id: re.search(str(register_id) , intro_text)).isna()
    df_filtered = knf_df[~ does_not_id_exist]

    fund_name = df_filtered['subfundusz'].sort_values(key=lambda x: x.str.len(), ascending=False).iloc[0]

    return fund_name

In [227]:
def retrive_organisation_name(knf_df: 'pd.DataFrame', intro_text: str) -> str:

    does_not_name_exist = knf_df['towarzystwo'].apply(lambda register_id: re.search(str(register_id) , intro_text)).isna()
    df_filtered = knf_df[~ does_not_id_exist]

    fund_name = df_filtered['towarzystwo'].sort_values(key=lambda x: x.str.len(), ascending=False).iloc[0]

    return fund_name

In [171]:
df.intro.apply(lambda x : retrive_register_id(knf_df, x))

0    182
Name: intro, dtype: int64

In [209]:
df.intro.apply(lambda x : retrive_fund_name(knf_df, x))

0    Millennium Specjalistyczny Fundusz Inwestycyjn...
Name: intro, dtype: object

In [211]:
df.intro.apply(lambda x : retrive_subfund_name(knf_df, x))

0    Subfundusz Obligacji Globalnych
Name: intro, dtype: object

In [213]:
df.intro.apply(lambda x : retrive_organisation_name(knf_df, x))

0    MILLENNIUM Towarzystwo Funduszy Inwestycyjnych...
Name: intro, dtype: object